In [23]:
import re
import numpy as np
import pandas as pd
import matplotlib as plt
import re
import json
from konlpy.tag import Okt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

DATA_IN_PATH = './data/'

파일 불러오기

In [24]:
Crawl_Data = pd.read_csv('data/KakaoBank_Reveiw_Crawl.csv', encoding='utf-8')

In [25]:
Crawl_Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3800 entries, 0 to 3799
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  3800 non-null   int64 
 1   id          3800 non-null   int64 
 2   date        3800 non-null   int64 
 3   dateYear    3800 non-null   int64 
 4   dateMonth   3800 non-null   int64 
 5   dateDay     3800 non-null   int64 
 6   rating      3800 non-null   int64 
 7   userName    3800 non-null   object
 8   content     3800 non-null   object
dtypes: int64(7), object(2)
memory usage: 267.3+ KB


중복값 확인 및 처리

In [26]:
Crawl_Data['content'].nunique()

3746

In [27]:
Crawl_Data.drop_duplicates(subset=['content'], inplace=True)

Crawl_Data.drop(['Unnamed: 0', 'id', 'date', 'userName'], axis=1, inplace=True)

Test_Data = Crawl_Data

전처리 함수

In [28]:
def preprocessing(review, okt, remove_stopwords = False, stop_words = []):
    # 함수의 인자는 다음과 같다.
    # review : 전처리할 텍스트
    # okt : okt 객체를 반복적으로 생성하지 않고 미리 생성후 인자로 받는다.
    # remove_stopword : 불용어를 제거할지 선택 기본값은 False
    # stop_word : 불용어 사전은 사용자가 직접 입력해야함 기본값은 비어있는 리스트
    
    # 1. 한글 및 공백을 제외한 문자 모두 제거.
    review_text = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", "", review)
    
    # 2. okt 객체를 활용해서 형태소 단위로 나눈다.
    word_review = okt.morphs(review_text, stem=True)
    
    if remove_stopwords:
        
        # 불용어 제거(선택적)
        word_review = [token for token in word_review if not token in stop_words]
        
   
    return word_review

In [29]:
okt=Okt()
processed_data=[]

test_data = []

for review in Crawl_Data['content']:
    processed_data.append(preprocessing(review, okt, remove_stopwords=True, stop_words=[]))
    
for review in Test_Data['content']:
    test_data.append(preprocessing(review, okt, remove_stopwords=True, stop_words=[]))

토큰화

In [30]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(processed_data)
Train_sequences=tokenizer.texts_to_sequences(processed_data)
Test_sequences=tokenizer.texts_to_sequences(test_data)

# 단어 사전 형태
word_vocab=tokenizer.word_index

# 문장 최대 길이
MAX_SEQUENCE_LENGTH=8

# 학습 데이터를 벡터화
Train_inputs=pad_sequences(Train_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
# 학습 데이터의 라벨
Train_labels=np.array([Crawl_Data['rating']])

# 평가 데이터를 벡터화
Test_inputs=pad_sequences(Test_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
# 평가 데이터의 라벨
Test_labels=np.array([Test_Data['rating']])

In [34]:
Train_i = pd.DataFrame(Train_inputs)
Train_l = pd.DataFrame(Train_labels)
Test_i = pd.DataFrame(Test_inputs)
Test_l = pd.DataFrame(Test_labels)

Train_i.to_csv('./data/Train_inputs.csv', encoding='utf-8')
Train_l.to_csv('./data/Train_labels.csv', encoding='utf-8')
Test_i.to_csv('./data/Test_inputs.csv', encoding='utf-8')
Test_l.to_csv('./data/Test_labels.csv', encoding='utf-8')

data_configs = {}

data_configs['vocab_size'] = word_vocab

# 데이터 사전을 json 형태로 저장
json.dump(data_configs, open(DATA_IN_PATH+'data_configs.json','w'), ensure_ascii=False)